In [610]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [611]:
from extract.polocm import AP, Hypothesis, POLOCM
from traces import *
from convertor import TopoConvertor
from observation import PartialOrderedActionObservation, ObservedPartialOrderTraceList
from extract.model import *
import pulp as pl
from tabulate import tabulate

In [612]:
objects = {
    "A": PlanningObject("unknown", "A"),
    "B": PlanningObject("unknown", "B"),
    "C": PlanningObject("unknown", "C"),
}

actions = {
    "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
    "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
    "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
    "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
    "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
    "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
    "stackAB": Action("stack", [objects["A"], objects["B"]]),
    "stackBA": Action("stack", [objects["B"], objects["A"]]),
    "stackAC": Action("stack", [objects["A"], objects["C"]]),
    "stackCA": Action("stack", [objects["C"], objects["A"]]),
    "stackBC": Action("stack", [objects["B"], objects["C"]]),
    "stackCB": Action("stack", [objects["C"], objects["B"]]),
    "putdownA": Action("putdown", [objects["A"]]),
    "putdownB": Action("putdown", [objects["B"]]),
    "putdownC": Action("putdown", [objects["C"]]),
    "pickupA": Action("pickup", [objects["A"]]),
    "pickupB": Action("pickup", [objects["B"]]),
    "pickupC": Action("pickup", [objects["C"]]),
}

traces = TraceList(
    [
        Trace(
            [
                Step(State({}), actions["unstackAB"], 0),
                Step(State({}), actions["putdownA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["stackBC"], 3),
                Step(State({}), actions["unstackBC"], 4),
                Step(State({}), actions["stackBC"], 5),
                Step(State({}), actions["pickupA"], 6),
                Step(State({}), actions["stackAB"], 7),
            ]
        ),
        Trace(
            [
                Step(State({}),actions['unstackCB'], 0),
                Step(State({}),actions['putdownC'], 1),
                Step(State({}),actions['unstackBA'], 2),
                Step(State({}),actions['putdownB'], 3),
                Step(State({}),actions['pickupB'], 4),
                Step(State({}),actions['stackBC'], 5),
                Step(State({}),actions['pickupA'], 6),
                Step(State({}),actions['stackAB'], 7),

            ]
        ),
        Trace(
            [
                Step(State({}), actions["unstackCB"], 0),
                Step(State({}), actions["stackCA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["putdownB"], 3),
                Step(State({}), actions["pickupB"], 4),
                Step(State({}), actions["stackBC"], 5),
            ]
        ),
        
    ]
)


In [613]:
convertor = TopoConvertor("flex", True, True, 2)
po_tracelist = traces.topo(convertor, 0.3)
print(tabulate(po_tracelist[0].comparable_matrix, headers='keys', tablefmt="fancy_grid"))
obs_tracelist = po_tracelist.tokenize(PartialOrderedActionObservation, ObservedPartialOrderTraceList)

╒════════════╤══════════════╤══════════════╤═════════════╤════════════╤══════════════╤════════════╤═════════════╤════════════╕
│            │   unstack[0] │   putdown[1] │   pickup[2] │   stack[3] │   unstack[4] │   stack[5] │   pickup[6] │   stack[7] │
╞════════════╪══════════════╪══════════════╪═════════════╪════════════╪══════════════╪════════════╪═════════════╪════════════╡
│ unstack[0] │          nan │            1 │           1 │        nan │            1 │          1 │           1 │          1 │
├────────────┼──────────────┼──────────────┼─────────────┼────────────┼──────────────┼────────────┼─────────────┼────────────┤
│ putdown[1] │          nan │          nan │           1 │        nan │            1 │          1 │           1 │          1 │
├────────────┼──────────────┼──────────────┼─────────────┼────────────┼──────────────┼────────────┼─────────────┼────────────┤
│ pickup[2]  │          nan │          nan │         nan │        nan │          nan │          1 │         nan

In [614]:
sorts = POLOCM._get_sorts(obs_tracelist)
print(f"Sorts:\n{sorts}", end="\n\n")

Sorts:
{'C': 1, 'B': 1, 'A': 1, 'zero': 0}



In [615]:
obj_PO_trace_overall, trace_PO_matrix_overall, obj_trace_PO_matrix_overall, obj_trace_FO_matrix_overall, sort_aps = POLOCM.polocm_step1(obs_tracelist, sorts, True)

# Step 1

## Obj PO traces:


### Trace **0**


+-------------+--------------+--------------+--------------+
| zero zero   | unknown A    | unknown B    | unknown C    |
|-------------+--------------+--------------+--------------|
| [1]putdown  | [1]putdown.1 | [7]stack.2   | [3]stack.2   |
| [7]stack    | [7]stack.1   | [0]unstack.2 | [4]unstack.2 |
| [0]unstack  | [0]unstack.1 | [2]pickup.1  | [5]stack.2   |
| [2]pickup   | [6]pickup.1  | [3]stack.1   |              |
| [6]pickup   |              | [4]unstack.1 |              |
| [3]stack    |              | [5]stack.1   |              |
| [4]unstack  |              |              |              |
| [5]stack    |              |              |              |
+-------------+--------------+--------------+--------------+


### Trace **1**


...

╒══════════════╤════════════════╤══════════════╤══════════════╕
│              │   [0]unstack.1 │   [1]stack.1 │   [5]stack.2 │
╞══════════════╪════════════════╪══════════════╪══════════════╡
│ [0]unstack.1 │            nan │            1 │            0 │
├──────────────┼────────────────┼──────────────┼──────────────┤
│ [1]stack.1   │              0 │          nan │            1 │
├──────────────┼────────────────┼──────────────┼──────────────┤
│ [5]stack.2   │              0 │            0 │          nan │
╘══════════════╧════════════════╧══════════════╧══════════════╛


#### Object B


╒══════════════╤════════════════╤═══════════════╤══════════════╤════════════════╤═══════════════╕
│              │   [0]unstack.2 │   [4]pickup.1 │   [5]stack.1 │   [3]putdown.1 │   [2]pickup.1 │
╞══════════════╪════════════════╪═══════════════╪══════════════╪════════════════╪═══════════════╡
│ [0]unstack.2 │            nan │             0 │          nan │            nan │           nan │
├──────────────┼────────────────┼───────────────┼──────────────┼────────────────┼───────────────┤
│ [4]pickup.1  │              0 │           nan │          nan │            nan │             0 │
├──────────────┼────────────────┼───────────────┼──────────────┼────────────────┼───────────────┤
│ [5]stack.1   │              0 │           nan │          nan │            nan │           nan │
├──────────────┼────────────────┼───────────────┼──────────────┼────────────────┼───────────────┤
│ [3]putdown.1 │              0 │           nan │          nan │            nan │           nan │
├──────────────┼────

#### Object A


╒════════════╤══════════════╕
│            │   [1]stack.2 │
╞════════════╪══════════════╡
│ [1]stack.2 │          nan │
╘════════════╧══════════════╛


In [616]:
prob, PO_vars_overall, PO_matrix_with_vars = POLOCM.polocm_step2(trace_PO_matrix_overall, obj_trace_PO_matrix_overall, debug=True)

# Step 2

## PO vars

### Trace 0

{([2]pickup, [6]pickup): trace_0__2_pickup~_6_pickup,
 ([3]stack, [0]unstack): trace_0__3_stack~_0_unstack,
 ([1]putdown, [3]stack): trace_0__1_putdown~_3_stack,
 ([7]stack, [6]pickup): trace_0__7_stack~_6_pickup,
 ([2]pickup, [4]unstack): trace_0__2_pickup~_4_unstack,
 ([6]pickup, [2]pickup): trace_0__6_pickup~_2_pickup,
 ([2]pickup, [3]stack): trace_0__2_pickup~_3_stack,
 ([3]stack, [2]pickup): trace_0__3_stack~_2_pickup,
 ([0]unstack, [3]stack): trace_0__0_unstack~_3_stack,
 ([4]unstack, [7]stack): trace_0__4_unstack~_7_stack,
 ([3]stack, [1]putdown): trace_0__3_stack~_1_putdown,
 ([7]stack, [4]unstack): trace_0__7_stack~_4_unstack,
 ([6]pickup, [5]stack): trace_0__6_pickup~_5_stack,
 ([5]stack, [6]pickup): trace_0__5_stack~_6_pickup,
 ([4]unstack, [2]pickup): trace_0__4_unstack~_2_pickup,
 ([6]pickup, [7]stack): trace_0__6_pickup~_7_stack,
 ([7]stack, [5]stack): trace_0__7_stack~_5_stack,
 ([5]stack, [7]stack): trace_0__5_stack~_7_stack}


### Trace 1

...

╒══════════════╤════════════════╤═════════════════════════╤════════════════════════╤═════════════════════════╤═════════════════════════╕
│              │   [0]unstack.2 │ [4]pickup.1             │ [5]stack.1             │ [3]putdown.1            │ [2]pickup.1             │
╞══════════════╪════════════════╪═════════════════════════╪════════════════════════╪═════════════════════════╪═════════════════════════╡
│ [0]unstack.2 │            nan │ 1                       │ 1                      │ 1                       │ 1                       │
├──────────────┼────────────────┼─────────────────────────┼────────────────────────┼─────────────────────────┼─────────────────────────┤
│ [4]pickup.1  │              0 │ nan                     │ ([4]pickup, [5]stack)  │ ([4]pickup, [3]putdown) │ 0                       │
├──────────────┼────────────────┼─────────────────────────┼────────────────────────┼─────────────────────────┼─────────────────────────┤
│ [5]stack.1   │              0 │ ([5]sta

#### Obj A

╒════════════╤══════════════╕
│            │   [1]stack.2 │
╞════════════╪══════════════╡
│ [1]stack.2 │          nan │
╘════════════╧══════════════╛


## PO Constraints

╒══════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════╕
│ Trace 0                                                                                  │ Trace 1                                                                                            │ Trace 2                                                                                      │
╞══════════════════════════════════════════════════════════════════════════════════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════╪══════════════════════════════════════════════════════════════════════════════════════════════╡
│ 1 >= 1 + trace_0__7_stack~_6_pickup - 1                                                  │ trace_1__0_unstack~_1_putdown >= 1 + tra

In [617]:
prob, FO_vars_overall, FO_matrix_with_vars, varname_lookup = POLOCM.polocm_step3(prob,PO_vars_overall, PO_matrix_with_vars, obj_trace_FO_matrix_overall, debug=True)

# Step 3

## FO vars

### Trace 0

╒══════════════════════════╤═════════════════════════════╤══════════════════════════════╤═════════════╕
│ zero zero                │ unknown A                   │ unknown B                    │ unknown C   │
╞══════════════════════════╪═════════════════════════════╪══════════════════════════════╪═════════════╡
│ ([1]putdown, [2]pickup)  │ ([1]putdown.1, [7]stack.1)  │ ([7]stack.2, [4]unstack.1)   │             │
├──────────────────────────┼─────────────────────────────┼──────────────────────────────┼─────────────┤
│ ([1]putdown, [3]stack)   │ ([1]putdown.1, [6]pickup.1) │ ([7]stack.2, [5]stack.1)     │             │
├──────────────────────────┼─────────────────────────────┼──────────────────────────────┼─────────────┤
│ ([1]putdown, [4]unstack) │ ([7]stack.1, [6]pickup.1)   │ ([0]unstack.2, [2]pickup.1)  │             │
├──────────────────────────┼─────────────────────────────┼──────────────────────────────┼─────────────┤
│ ([7]stack, [6]pickup)    │ ([6]pickup.1, [7]stack.1)   │ ([0]u

### Trace 1

...

╒══════════════╤════════════════╤═════════════════════════════╤════════════════════════════╤══════════════════════════════╤═════════════════════════════╕
│              │   [0]unstack.2 │ [4]pickup.1                 │ [5]stack.1                 │ [3]putdown.1                 │ [2]pickup.1                 │
╞══════════════╪════════════════╪═════════════════════════════╪════════════════════════════╪══════════════════════════════╪═════════════════════════════╡
│ [0]unstack.2 │            nan │ 0                           │ ([0]unstack.2, [5]stack.1) │ ([0]unstack.2, [3]putdown.1) │ ([0]unstack.2, [2]pickup.1) │
├──────────────┼────────────────┼─────────────────────────────┼────────────────────────────┼──────────────────────────────┼─────────────────────────────┤
│ [4]pickup.1  │              0 │ nan                         │ ([4]pickup.1, [5]stack.1)  │ ([4]pickup.1, [3]putdown.1)  │ 0                           │
├──────────────┼────────────────┼─────────────────────────────┼─────────────

#### Obj A

╒════════════╤══════════════╕
│            │   [1]stack.2 │
╞════════════╪══════════════╡
│ [1]stack.2 │          nan │
╘════════════╧══════════════╛


## FO Constraints

╒════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════

In [618]:
prob, sort_transition_matrix, sort_AP_vars, varname_lookup = POLOCM.polocm_step4(prob, sort_aps, sorts,FO_vars_overall, FO_matrix_with_vars,varname_lookup, debug=True)

# Step 4

## AP vars

╒════════════════════════╤════════════════════════╕
│ 0                      │ 1                      │
╞════════════════════════╪════════════════════════╡
│ (unstack.0, unstack.0) │ (unstack.1, stack.2)   │
├────────────────────────┼────────────────────────┤
│ (unstack.0, pickup.0)  │ (unstack.1, pickup.1)  │
├────────────────────────┼────────────────────────┤
│ (unstack.0, putdown.0) │ (unstack.2, unstack.1) │
├────────────────────────┼────────────────────────┤
│ (pickup.0, unstack.0)  │ (unstack.2, pickup.1)  │
├────────────────────────┼────────────────────────┤
│ (pickup.0, pickup.0)   │ (unstack.2, stack.1)   │
├────────────────────────┼────────────────────────┤
│ (pickup.0, putdown.0)  │ (unstack.2, putdown.1) │
├────────────────────────┼────────────────────────┤
│ (pickup.0, stack.0)    │ (stack.2, unstack.1)   │
├────────────────────────┼────────────────────────┤
│ (putdown.0, unstack.0) │ (stack.2, stack.1)     │
├────────────────────────┼────────────────────────┤
│ (putdown.0

## AP matrix with vars

### Sort 0

╒═══════════╤════════════════════════╤═══════════════════════╤════════════════════════╤══════════════════════╕
│           │ unstack.0              │ pickup.0              │ putdown.0              │ stack.0              │
╞═══════════╪════════════════════════╪═══════════════════════╪════════════════════════╪══════════════════════╡
│ unstack.0 │ (unstack.0, unstack.0) │ (unstack.0, pickup.0) │ (unstack.0, putdown.0) │ 1                    │
├───────────┼────────────────────────┼───────────────────────┼────────────────────────┼──────────────────────┤
│ pickup.0  │ (pickup.0, unstack.0)  │ (pickup.0, pickup.0)  │ (pickup.0, putdown.0)  │ (pickup.0, stack.0)  │
├───────────┼────────────────────────┼───────────────────────┼────────────────────────┼──────────────────────┤
│ putdown.0 │ (putdown.0, unstack.0) │ (putdown.0, pickup.0) │ (putdown.0, putdown.0) │ (putdown.0, stack.0) │
├───────────┼────────────────────────┼───────────────────────┼────────────────────────┼──────────────────────┤
│

### Sort 1

╒═══════════╤════════════════════════╤══════════════════════╤══════════════════════╤═══════════════════════╤══════════════════════╤════════════════════════╕
│           │ unstack.1              │ unstack.2            │ stack.2              │ pickup.1              │ stack.1              │ putdown.1              │
╞═══════════╪════════════════════════╪══════════════════════╪══════════════════════╪═══════════════════════╪══════════════════════╪════════════════════════╡
│ unstack.1 │ nan                    │ nan                  │ (unstack.1, stack.2) │ (unstack.1, pickup.1) │ 1                    │ 1                      │
├───────────┼────────────────────────┼──────────────────────┼──────────────────────┼───────────────────────┼──────────────────────┼────────────────────────┤
│ unstack.2 │ (unstack.2, unstack.1) │ nan                  │ 1                    │ (unstack.2, pickup.1) │ (unstack.2, stack.1) │ (unstack.2, putdown.1) │
├───────────┼────────────────────────┼────────────────────

## AP Constraints

╒═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ 0                                                                                                                                                                                                                                                                                                                                   │ 1                                                                                                    

In [619]:
solver = pl.PULP_CBC_CMD(msg=True)
solution = POLOCM.polocm_step5(prob, solver,sort_AP_vars, True)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/uto/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bc3946f2dcfb47c3aedf5c26abf5d5ef-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/bc3946f2dcfb47c3aedf5c26abf5d5ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2287 COLUMNS
At line 8448 RHS
At line 10731 BOUNDS
At line 11235 ENDATA
Problem MODEL has 2282 rows, 503 columns and 5118 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10.8333 - 0.02 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 501 strengthened rows, 63 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 200 strengthened rows, 14 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 60 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightene

In [620]:
FO, AP = POLOCM.polocm_step6(PO_matrix_with_vars, PO_vars_overall,FO_matrix_with_vars, FO_vars_overall, sort_transition_matrix, sort_AP_vars, solution, debug=True)

## Solution FO matrix

### Trace 0

#### Obj zero

╒════════════╤══════════════╤════════════╤══════════════╤═════════════╤═════════════╤════════════╤══════════════╤════════════╕
│            │   [1]putdown │   [7]stack │   [0]unstack │   [2]pickup │   [6]pickup │   [3]stack │   [4]unstack │   [5]stack │
╞════════════╪══════════════╪════════════╪══════════════╪═════════════╪═════════════╪════════════╪══════════════╪════════════╡
│ [1]putdown │          nan │          0 │            0 │           1 │           0 │          0 │            0 │          0 │
├────────────┼──────────────┼────────────┼──────────────┼─────────────┼─────────────┼────────────┼──────────────┼────────────┤
│ [7]stack   │            0 │        nan │            0 │           0 │           0 │          0 │            0 │          0 │
├────────────┼──────────────┼────────────┼──────────────┼─────────────┼─────────────┼────────────┼──────────────┼────────────┤
│ [0]unstack │            1 │          0 │          nan │           0 │           0 │          0 │            0

#### Obj A

...

## Solution AP matrix

### Sort 0

╒═══════════╤═════════════╤════════════╤═════════════╤═══════════╕
│           │   unstack.0 │   pickup.0 │   putdown.0 │   stack.0 │
╞═══════════╪═════════════╪════════════╪═════════════╪═══════════╡
│ unstack.0 │           0 │          0 │           1 │         1 │
├───────────┼─────────────┼────────────┼─────────────┼───────────┤
│ pickup.0  │           0 │          0 │           0 │         1 │
├───────────┼─────────────┼────────────┼─────────────┼───────────┤
│ putdown.0 │           1 │          1 │           0 │         0 │
├───────────┼─────────────┼────────────┼─────────────┼───────────┤
│ stack.0   │           1 │          1 │           1 │         0 │
╘═══════════╧═════════════╧════════════╧═════════════╧═══════════╛


### Sort 1

╒═══════════╤═════════════╤═════════════╤═══════════╤════════════╤═══════════╤═════════════╕
│           │   unstack.1 │   unstack.2 │   stack.2 │   pickup.1 │   stack.1 │   putdown.1 │
╞═══════════╪═════════════╪═════════════╪═══════════╪════════════╪═══════════╪═════════════╡
│ unstack.1 │         nan │         nan │         0 │          0 │         1 │           1 │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼───────────┼─────────────┤
│ unstack.2 │           1 │         nan │         1 │          1 │         0 │           0 │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼───────────┼─────────────┤
│ stack.2   │           0 │           1 │       nan │        nan │         0 │         nan │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼───────────┼─────────────┤
│ pickup.1  │           0 │         nan │         0 │          0 │         1 │           0 │
├───────────┼─────────────┼─────────────┼───────────┼────────────┼────